In [1]:
import pandas as pd
import requests
import io
import os

# URLs oficiales (verificadas)
URL_DEMO = "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/DEMO_J.XPT"
URL_CBC = "https://wwwn.cdc.gov/Nchs/Nhanes/2017-2018/CBC_J.XPT"

# Definir ruta de datos relativa
DATA_DIR = "../data"
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

def download_and_read(url, name):
    print(f"⬇️ Descargando {name}...")
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        
        if response.status_code != 200:
            print(f"❌ Error HTTP {response.status_code} en {name}")
            return None
            
        if b"<!DOCTYPE html>" in response.content[:100] or b"<html" in response.content[:100]:
            print(f"❌ Error: El servidor devolvió una página web, no el archivo de datos para {name}.")
            return None

        return pd.read_sas(io.BytesIO(response.content), format='xport')

    except Exception as e:
        print(f"❌ Falló la carga de {name}: {e}")
        return None

def main():
    # 1. Intentar descarga automática
    df_demo = download_and_read(URL_DEMO, "Demografía")
    df_cbc = download_and_read(URL_CBC, "Laboratorio")
    
    # 2. PLAN DE EMERGENCIA
    if df_demo is None or df_cbc is None:
        print("\n" + "="*50)
        print("🚨 LA DESCARGA AUTOMÁTICA FALLÓ (Bloqueo de red/CDC)")
        print("⚡ ACCIÓN MANUAL REQUERIDA:")
        print("1. Descarga estos 2 archivos manualmente.")
        print("2. Guárdalos en la carpeta 'data' del proyecto.")
        return

    # 3. Fusión y Limpieza
    print("✅ Datos cargados. Procesando...")
    df_merged = pd.merge(df_demo, df_cbc, on="SEQN", how="inner")
    
    column_mapping = {
        'SEQN': 'ID', 'RIAGENDR': 'Sex', 'RIDAGEYR': 'Age',
        'LBXWBCSI': 'WBC', 'LBXNEPCT': 'NEU_percent', 'LBXLYPCT': 'LYM_percent',
        'LBXRBCSI': 'RBC', 'LBXHGB': 'HGB', 'LBXHCT': 'HCT',
        'LBXMCV': 'MCV', 'LBXMCH': 'MCH', 'LBXPLTSI': 'PLT'
    }
    
    avail_cols = [c for c in column_mapping.keys() if c in df_merged.columns]
    df_clean = df_merged[avail_cols].rename(columns=column_mapping)
    
    if 'WBC' in df_clean.columns:
        if 'NEU_percent' in df_clean.columns:
            df_clean['NEU'] = df_clean['WBC'] * (df_clean['NEU_percent'] / 100)
        if 'LYM_percent' in df_clean.columns:
            df_clean['LYM'] = df_clean['WBC'] * (df_clean['LYM_percent'] / 100)
            
    targets = ['HGB', 'HCT', 'RBC', 'MCV', 'MCH', 'WBC', 'PLT']
    final_targets = [t for t in targets if t in df_clean.columns]
    df_clean = df_clean.dropna(subset=final_targets)
    
    # --- RUTA CORREGIDA ---
    output_path = os.path.join(DATA_DIR, "nhanes_raw_data.csv")
    df_clean.to_csv(output_path, index=False)
    print(f"🎉 ÉXITO TOTAL: Dataset de {df_clean.shape[0]} pacientes guardado en: {output_path}")

if __name__ == "__main__":
    main()

⬇️ Descargando Demografía...
❌ Error: El servidor devolvió una página web, no el archivo de datos para Demografía.
⬇️ Descargando Laboratorio...
❌ Error: El servidor devolvió una página web, no el archivo de datos para Laboratorio.

🚨 LA DESCARGA AUTOMÁTICA FALLÓ (Bloqueo de red/CDC)
⚡ ACCIÓN MANUAL REQUERIDA:
1. Descarga estos 2 archivos manualmente.
2. Guárdalos en la carpeta 'data' del proyecto.
